<a href="https://colab.research.google.com/github/OfficalOffical/BasicTranslator/blob/master/Copy_of_modelTraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.src.regularizers import l2
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt





In [6]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from scipy.stats import zscore
import re


def work_on_outliers(input_df,inf_threshold,sup_threshold, feature_number,visualize):
    # WORKING ON OUTLIERS

    # Working on negpmax feature (the only one that is showing outliers)
    features = [f for f in input_df.columns[feature_number::4]]
    if visualize:
        fig, axes = plt.subplots(4, 3, figsize=(15, 10))  # Adjust the figsize as needed

        # Flatten the 2D array of subplots
        axes = axes.flatten()

    output_df=input_df.copy()

    for i, feature in enumerate(features):
        Q1 = input_df[feature].quantile(0.25)
        Q3 = input_df[feature].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - inf_threshold * IQR

        if feature_number==3:
            upper_bound=0
        else:
            upper_bound=Q3 + sup_threshold * IQR




        # removing too  small values and positive values for negpmax
        #removing too large values for rms

        outliers = output_df[(output_df[feature] < lower_bound) | (output_df[feature] > upper_bound)]



        output_df = output_df[(output_df[feature] > lower_bound) & (output_df[feature] < upper_bound)]
        if visualize:
            axes[i].scatter(outliers["x"], outliers["y"], c=outliers[feature], cmap='viridis')  # Adjust the colormap as needed
            axes[i].set_title(f'REMOVED OUTLIERS: {feature}')
            axes[i].set_xlabel("X-axis")
            axes[i].set_ylabel("Y-axis")

    if visualize:
        plt.tight_layout()
        plt.show()


    if visualize:

        # let's see if the representation is working better after removing the outliers '
        random_10_percent = output_df.sample(frac=0.1, random_state=42)

        fig, axes = plt.subplots(4, 3, figsize=(15, 10))  # Adjust the figsize as needed
        fig.suptitle("RESULT AFTER REMOVING OUTLIERS (ON A 10% SAMPLE) ", fontsize=16)
        # Flatten the 2D array of subplots
        axes = axes.flatten()

        for i, feature in enumerate(features):
            axes[i].scatter(random_10_percent["x"], random_10_percent["y"], c=random_10_percent[feature],
                            cmap='viridis',label=feature)  # Adjust the colormap as needed
            axes[i].set_title(f'scatter plot: {feature}')
            axes[i].set_xlabel("X-axis")
            axes[i].set_ylabel("Y-axis")
            axes[i].legend(loc='best')  # Add legend

        # Adjust layout to prevent overlap
        plt.tight_layout()
        plt.show()


    return output_df


#MAIN CODE
def pre_process(df, normalize=False, see_graphs=False):
    #REMOVING NOISE COLUMNS

    #working on tmax features, which seem to be the most meaningful for removing the noise sensors
    z_scores = pd.DataFrame(zscore(df), columns=df.columns[5::5])
    outlier_threshold = 3
    outliers = (z_scores.abs() > outlier_threshold)

    #noise sensors have a random distribution of tmax, almost all  the values are contained
    #in the value of 3*dev_std. We order by increasing order of outliers and remove the columns
    #that have less outliers (random distribution)

    percentage_outliers = (outliers.sum() / len(df)) * 100
    sorted_columns = percentage_outliers.sort_values()
    numbers_to_remove = [re.search(r'\[(\d+)\]', column).group(1) for column in sorted_columns[:6].index]
    columns_to_drop = [column for column in df.columns if
                       any(re.search(rf'\[{number}\]', column) for number in numbers_to_remove)]

    df_clean = df.drop(columns=columns_to_drop)

    # the feature describing the area and the feature describing the pmax are highly correlated,
    # we can remove the area feature.

    features_area = [f for f in df_clean.columns[4::5]]

    df_clean = df_clean.drop(columns=features_area)

    #OUTLIERS

    df_clean_out_negpmax=work_on_outliers(df_clean,25,0,3,see_graphs) #3 means negpmax

    #we can work also on the rms

    df_clean_out_rms = work_on_outliers(df_clean, 1.5,1.5, 5,see_graphs)  # 5 means rms

    # SPLITTING AND NORMALIZING IF REQUIRED
    df_final=df_clean_out_rms.copy()
    X=df_final.iloc[:,2:]
    y=df_final.iloc[:,:2]

    if normalize:
        scaler = StandardScaler()

        X_scaled = scaler.fit_transform(X)
    else:
        X_scaled= X.copy()



    return df_final













In [7]:
from tensorflow.keras.losses import Loss
import tensorflow.keras.backend as K

def euclidean_distance_loss(y_true, y_pred):
    return K.sqrt(K.sum(K.square(y_pred - y_true), axis=-1))

def euclidean_distance_error(y_true, y_pred):
    """
    Calculate the mean Euclidean distance error between true and predicted 2D points.
    :param y_true: Array of true values with shape (n_samples, 2).
    :param y_pred: Array of predicted values with shape (n_samples, 2).
    :return: Mean Euclidean distance error.
    """
    return np.mean(np.sqrt(np.sum(np.square(y_true - y_pred), axis=1)))



In [24]:
df = pd.read_csv('development.csv')
df = pre_process(df)
print(df.shape)
#get random 20 k samples



# Prepare the dataset
X = df.iloc[:, 2:].values
Y = df.iloc[:, :2].values

(347342, 50)


In [54]:
# Split the dataset into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)



# Define the model
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='linear'))

# make the step of adam optimizer more smaller


# Compile the model
model.compile(optimizer='adam', loss='mse')

model.summary()

# Train the model
model.fit(X_train, Y_train, epochs=50, batch_size=32, verbose=1)  # Adjust epochs and batch_size as needed

# Evaluate the model
predictions = model.predict(X_test)
mse = mean_squared_error(Y_test, predictions)
print(f"Mean Squared Error: {mse}")
rmse = sqrt(mse)
print(f"Root Mean Squared Error: {rmse}")
euclidian= euclidean_distance_error(Y_test, predictions)
print(f"Euclidian distance Error: {euclidian}")

# Save the model for later use if needed
# model.save('your_model.h5')


Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_101 (Dense)           (None, 128)               6272      
                                                                 
 dense_102 (Dense)           (None, 64)                8256      
                                                                 
 dense_103 (Dense)           (None, 32)                2080      
                                                                 
 dense_104 (Dense)           (None, 2)                 66        
                                                                 
Total params: 16674 (65.13 KB)
Trainable params: 16674 (65.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/50
8684/8684 [==============================] - 21s 2ms/step - loss: 2997.4817
Epoch 2/50
8684/8684 [==============================] - 19s 2m

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Reshape the data to add a third dimension
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [26]:
model = Sequential([
    Conv1D(filters=128, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)),
    BatchNormalization(),
    Dropout(0.3),
    MaxPooling1D(pool_size=3),


    Conv1D(filters=64, kernel_size=2, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=3),

    Conv1D(filters=64, kernel_size=2, activation='relu'),
    BatchNormalization(),

    Conv1D(filters=32, kernel_size=2, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),

    Flatten(),
    Dense(128, activation= 'relu'),
    Dense(64, activation= 'relu'),
    Dense(32, activation= 'relu'),
    Dense(2, activation='linear')
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss=euclidean_distance_loss,  # Using Mean Squared Error
              metrics=['mae'])  # Mean Absolute Error as metric

# Summary of the Model
model.summary()

# Train the model
model.fit(X_train, Y_train, epochs=50, batch_size=32, verbose=1)

# Evaluate the model
predictions = model.predict(X_test)
mse = mean_squared_error(Y_test, predictions)
print(f"Mean Squared Error: {mse}")
rmse = sqrt(mse)
print(f"Root Mean Squared Error: {rmse}")
euclidian= euclidean_distance_error(Y_test, predictions)
print(f"Euclidian distance Error: {euclidian}")

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_45 (Conv1D)          (None, 47, 128)           384       
                                                                 
 batch_normalization_42 (Ba  (None, 47, 128)           512       
 tchNormalization)                                               
                                                                 
 dropout_21 (Dropout)        (None, 47, 128)           0         
                                                                 
 max_pooling1d_33 (MaxPooli  (None, 15, 128)           0         
 ng1D)                                                           
                                                                 
 conv1d_46 (Conv1D)          (None, 14, 64)            16448     
                                                                 
 batch_normalization_43 (Ba  (None, 14, 64)          